# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [2]:
url = "https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/whpps_120717.csv"
health = pd.read_csv(url, sep="~")

In [3]:
rows = [13, 27, 101]
health.loc[rows,:]

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,"Finalwt_worksite,,,,"
13,3,1.0,2.0,3.0,2.0,1.0,1.0,3.0,3.0,1.0,...,NaN,NaN,1.0,1437.0,4.0,6.0,7.0,3.0,0.0,"47.793940929,,,,"
27,1,3.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,1.0,2501.0,2.0,4.0,7.0,8.0,0.0,"47.793940929,,,,"
101,2,1.0,1.0,3.0,2.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,2.0,12636.0,4.0,6.0,7.0,4.0,0.0,"47.793940929,,,,"


## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [4]:
cols2 = ["Industry", "Size", "OC3", "HI1", "HI2", "HI3", "CP1", "WL6"]
list1 = [x for x in health.columns if x.startswith("WD")]

clean_df = health[cols2 + list1]
clean_df.head(5)

,Industry,Size,OC3,HI1,HI2,HI3,CP1,WL6,WD1_1,WD1_2,WD2,WD3,WD4,WD5,WD6,WD7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [5]:
clean_df = clean_df.rename({"Industry": "industry", "Size":"size", "OC3":"org_type", "HI1": "premium", 
             "HI2": "premium_change", "HI3": "pt_insurance", "CP1": "health_ed_program", 
             "WL6":"work_from_home", "WD1_1": "percent_under_30", "WD1_2": "percent_over_60",
            "WD2": "percent_female", "WD3": "percent_hourly", "WD4" : "percent_atypical_shifts", 
             "WD5": "percent_offsite", "WD6": "percent_unionized", "WD7": "percent_annual_turnover"}, axis=1)
clean_df.head()

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [6]:
# Each of these labels represents quite a few different industries
new_labels = {1.0: "Agriculture and Industrial Production",
              2.0: "Trade and Exchange",
              3.0: "Leisure and Hospitality",
              4.0: "Business Sales and Services",
              5.0: "Social Services",
              6.0: "Public Administration",
              7.0: "Hospitals"
             }

In [130]:
# Each of these labels represents quite a few different industries
#new_labels = {1.0: "Agriculture, Forestry, Fishing and Hunting, Mining, Utilities, Construction, Manufacturing",
              # 2.0: "Wholesale Trade, Retail Trade, Transportation and Warehousing",
              # 3.0: "Arts, Entertainment, and Recreation; Accomodation and Food Services; Other Services",
              # 4.0: "Information; Finance and Insurance; Real Estate Rental and Leasing; Professional, Scientific, and Technical Services; Company and Enterprise Management; Administrative and Support and Waste Management and Remediation Services",
              # 5.0: "Educational Services, Healthcare and Social Assistance",
              # 6.0: "Public Administration",
              # 7.0: "Hospital Worksites"}

In [7]:
clean_df.industry = clean_df.industry.map(new_labels)

In [8]:
clean_df.head(5)

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
0,Hospitals,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospitals,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,Hospitals,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,Hospitals,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,Hospitals,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [9]:
clean_df["size"]= pd.cut(clean_df["size"], bins=[0,3,5,8], labels=("Small", "Medium", "Large"))
clean_df

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
0,Hospitals,Large,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospitals,Large,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,Hospitals,Large,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,Hospitals,Medium,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,Hospitals,Medium,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,Public Administration,Medium,4.0,1.0,3.0,1.0,1.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,Public Administration,Medium,4.0,2.0,3.0,1.0,1.0,2.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,Public Administration,Large,4.0,2.0,3.0,1.0,1.0,1.0,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,Public Administration,Large,4.0,2.0,3.0,1.0,2.0,99.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [10]:
replace_map = {"org_type": {1.0: "For profit, public",
                                     2.0: "For profit, private",
                                     3.0: "Non-profit",
                                     4.0: "State or local gov't",
                                     5.0: "Federal gov't",
                                     6.0: "Other",
                                     97.0: np.nan,
                                     98.0: np.nan,
                                     99.0: np.nan},
               "premium": {1.0: "Full coverage",
                                      2.0: "Partial coverage",
                                      3.0: "No coverage",
                                      97.0: np.nan,
                                      98.0: np.nan,
                                      99.0: np.nan},
               "premium_change": {1.0: "Larger",
                                               2.0: "Smaller",
                                               3.0: "About the same",
                                               96.0: np.nan,
                                               97.0: np.nan,
                                               98.0: np.nan,
                                               99.0: np.nan},
               "pt_insurance": {1.0: "Yes",
                                    2.0: "No",
                                    97.0: np.nan,
                                    98.0: np.nan,
                                    99.0: np.nan},
               "health_ed_program": {1.0: "Yes",
                                     2.0: "No",
                                     97.0: np.nan,
                                     98.0: np.nan},
               "work_from_home": {1.0: "Yes",
                                  2.0: "No",
                                  97.0: np.nan,
                                  98.0: np.nan,
                                  99.0: np.nan}}

In [11]:
org_df = clean_df.replace(replace_map)
org_df

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
0,Hospitals,Large,Non-profit,Partial coverage,Larger,No,Yes,Yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospitals,Large,Non-profit,Partial coverage,About the same,Yes,Yes,Yes,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,Hospitals,Large,Non-profit,Full coverage,About the same,Yes,Yes,Yes,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,Hospitals,Medium,"For profit, private",Full coverage,Smaller,Yes,No,No,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,Hospitals,Medium,Non-profit,Full coverage,About the same,Yes,Yes,Yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,Public Administration,Medium,State or local gov't,Full coverage,About the same,Yes,Yes,NaN,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2839,Public Administration,Medium,State or local gov't,Partial coverage,About the same,Yes,Yes,No,997.0,997.0,997.0,997.0,997.0,997.0,997.0,997.0
2840,Public Administration,Large,State or local gov't,Partial coverage,About the same,Yes,Yes,Yes,27.0,997.0,61.0,997.0,997.0,997.0,997.0,997.0
2841,Public Administration,Large,State or local gov't,Partial coverage,About the same,Yes,No,NaN,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [12]:
num_cols = ['percent_under_30', 'percent_over_60', 'percent_female', 'percent_hourly',
       'percent_atypical_shifts', 'percent_offsite', 'percent_unionized', 
       'percent_annual_turnover']

org_df[num_cols] = org_df[num_cols].replace([997.0, 998.0, 999.0], np.nan)
org_df.head(5)

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
0,Hospitals,Large,Non-profit,Partial coverage,Larger,No,Yes,Yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,Hospitals,Large,Non-profit,Partial coverage,About the same,Yes,Yes,Yes,NaN,NaN,90.0,90.0,NaN,NaN,0.0,NaN
2,Hospitals,Large,Non-profit,Full coverage,About the same,Yes,Yes,Yes,35.0,4.0,NaN,NaN,40.0,15.0,NaN,NaN
3,Hospitals,Medium,"For profit, private",Full coverage,Smaller,Yes,No,No,50.0,15.0,50.0,85.0,75.0,0.0,0.0,NaN
4,Hospitals,Medium,Non-profit,Full coverage,About the same,Yes,Yes,Yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [13]:
# alphabetize
org_df = org_df.sort_values(["industry", "size"])
# sort by under 30
org_df = org_df.sort_values(["percent_under_30"], ascending=False)
# show results
org_df.head()

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
900,Agriculture and Industrial Production,Small,NaN,No coverage,NaN,No,No,Yes,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0
2639,Leisure and Hospitality,Small,"For profit, private",No coverage,NaN,No,No,No,100.0,0.0,40.0,100.0,100.0,0.0,0.0,10.0
2034,Agriculture and Industrial Production,Small,"For profit, private",NaN,NaN,No,No,No,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN
554,Trade and Exchange,Small,"For profit, private",NaN,NaN,No,Yes,Yes,100.0,0.0,75.0,80.0,0.0,0.0,0.0,NaN
2398,Leisure and Hospitality,Small,NaN,NaN,NaN,No,NaN,No,100.0,0.0,90.0,100.0,0.0,0.0,NaN,NaN


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [14]:
# use .dropna() to drop NaN vals only within "Industry" column
org_df = org_df.dropna(subset=["industry"])
org_df.head()

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover
900,Agriculture and Industrial Production,Small,NaN,No coverage,NaN,No,No,Yes,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0
2639,Leisure and Hospitality,Small,"For profit, private",No coverage,NaN,No,No,No,100.0,0.0,40.0,100.0,100.0,0.0,0.0,10.0
2034,Agriculture and Industrial Production,Small,"For profit, private",NaN,NaN,No,No,No,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN
554,Trade and Exchange,Small,"For profit, private",NaN,NaN,No,Yes,Yes,100.0,0.0,75.0,80.0,0.0,0.0,0.0,NaN
2398,Leisure and Hospitality,Small,NaN,NaN,NaN,No,NaN,No,100.0,0.0,90.0,100.0,0.0,0.0,NaN,NaN


## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

In [15]:
org_df["gender_balance"] = pd.cut(clean_df.percent_female, 
                                    bins=[0,35, 65, 100], 
                                    labels= ("Mostly men", "Balanced", "Mostly women"))
org_df.head()

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover,gender_balance
900,Agriculture and Industrial Production,Small,NaN,No coverage,NaN,No,No,Yes,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0,Mostly women
2639,Leisure and Hospitality,Small,"For profit, private",No coverage,NaN,No,No,No,100.0,0.0,40.0,100.0,100.0,0.0,0.0,10.0,Balanced
2034,Agriculture and Industrial Production,Small,"For profit, private",NaN,NaN,No,No,No,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN,Balanced
554,Trade and Exchange,Small,"For profit, private",NaN,NaN,No,Yes,Yes,100.0,0.0,75.0,80.0,0.0,0.0,0.0,NaN,Mostly women
2398,Leisure and Hospitality,Small,NaN,NaN,NaN,No,NaN,No,100.0,0.0,90.0,100.0,0.0,0.0,NaN,NaN,Mostly women


## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

In [16]:
obj_names = ["industry", "org_type", "premium", "premium_change", 
             "pt_insurance", "health_ed_program", "work_from_home"]

org_df[obj_names] = org_df[obj_names].astype("category")

In [17]:
org_df.dtypes

industry                   category
size                       category
org_type                   category
premium                    category
premium_change             category
pt_insurance               category
health_ed_program          category
work_from_home             category
percent_under_30            float64
percent_over_60             float64
percent_female              float64
percent_hourly              float64
percent_atypical_shifts     float64
percent_offsite             float64
percent_unionized           float64
percent_annual_turnover     float64
gender_balance             category
dtype: object

## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

In [18]:
small_cats = org_df[(clean_df["size"] == "Small") & (org_df["work_from_home"] == "Yes")]

small_cats = small_cats.stb.freq(["premium"])
small_cats

,premium,count,percent,cumulative_count,cumulative_percent
0,Full coverage,324,46.285714,324,46.285714
1,Partial coverage,310,44.285714,634,90.571429
2,No coverage,66,9.428571,700,100.000000


## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

In [19]:
# export as csv
org_df.to_csv('wha.csv', index=False)

In [20]:
# import csv
wha_data = pd.read_csv("wha.csv")

In [21]:
wha_data

,industry,size,org_type,premium,premium_change,pt_insurance,health_ed_program,work_from_home,percent_under_30,percent_over_60,percent_female,percent_hourly,percent_atypical_shifts,percent_offsite,percent_unionized,percent_annual_turnover,gender_balance
0,Agriculture and Industrial Production,Small,NaN,No coverage,NaN,No,No,Yes,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0,Mostly women
1,Leisure and Hospitality,Small,"For profit, private",No coverage,NaN,No,No,No,100.0,0.0,40.0,100.0,100.0,0.0,0.0,10.0,Balanced
2,Agriculture and Industrial Production,Small,"For profit, private",NaN,NaN,No,No,No,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN,Balanced
3,Trade and Exchange,Small,"For profit, private",NaN,NaN,No,Yes,Yes,100.0,0.0,75.0,80.0,0.0,0.0,0.0,NaN,Mostly women
4,Leisure and Hospitality,Small,NaN,NaN,NaN,No,NaN,No,100.0,0.0,90.0,100.0,0.0,0.0,NaN,NaN,Mostly women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,Trade and Exchange,Large,"For profit, public",Full coverage,NaN,No,Yes,Yes,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN
2838,Trade and Exchange,Large,"For profit, private",Full coverage,About the same,No,Yes,Yes,NaN,NaN,NaN,NaN,0.0,NaN,NaN,22.0,NaN
2839,Trade and Exchange,Large,"For profit, public",Partial coverage,Larger,No,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2840,Trade and Exchange,Large,"For profit, private",Full coverage,Larger,NaN,NaN,Yes,NaN,1.0,NaN,90.0,75.0,1.0,90.0,NaN,NaN


In [22]:
db8 = "pandas_verify.db"

mod8db = sqlite3.connect(db8)
wha_data.to_sql("wha_data", mod8db, index=False, chunksize=1000, if_exists = "replace")

cursor8 = mod8db.cursor()

## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

In [23]:
more_info = wha_data[(wha_data["industry"] == "Hospitals") & (wha_data["premium_change"] == "Smaller")]

more_info = more_info[["size", "org_type", "premium_change", "percent_female"]]
more_info = more_info.rename({"premium_change":"insurance"}, axis="columns")
more_info

,size,org_type,insurance,percent_female
230,Medium,"For profit, private",Smaller,50.0
410,Medium,Non-profit,Smaller,NaN
727,Medium,Non-profit,Smaller,89.0
904,Large,Non-profit,Smaller,80.0
1459,Medium,NaN,Smaller,NaN
2131,Small,Non-profit,Smaller,75.0
2151,Medium,Non-profit,Smaller,NaN
2153,Medium,Non-profit,Smaller,NaN
2186,Medium,Non-profit,Smaller,65.0
2242,Large,Non-profit,Smaller,NaN


In [24]:
query = """
SELECT size, org_type, premium AS insurance, percent_female FROM wha_data
WHERE Industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;"""

cursor8.execute(query)
wha_d= cursor8.fetchall()
df = pd.DataFrame(wha_d)
df

,0,1,2,3
0,Medium,Non-profit,Full coverage,89.0
1,Large,Non-profit,Partial coverage,80.0
2,Large,Non-profit,Partial coverage,80.0
3,Small,Non-profit,Full coverage,75.0
4,Medium,Non-profit,Partial coverage,65.0
5,Medium,"For profit, private",Full coverage,50.0
6,Medium,Non-profit,Full coverage,NaN
7,Medium,None,Partial coverage,NaN
8,Medium,Non-profit,Partial coverage,NaN
9,Medium,Non-profit,Full coverage,NaN


## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

In [25]:
new_stuff = wha_data.groupby("industry").agg({"percent_female": "mean",
                                  "percent_under_30": "mean",
                                  "percent_over_60": "mean"}).sort_values("percent_female", ascending=False)

new_stuff

,percent_female,percent_under_30,percent_over_60
industry,,,
Social Services,80.657143,25.745665,11.349570
Hospitals,76.427027,27.213793,16.489655
Leisure and Hospitality,53.804416,38.566343,11.544872
Business Sales and Services,50.632184,23.821752,12.465465
Public Administration,39.056738,21.015625,15.015385
Trade and Exchange,32.657258,29.108696,12.584034
Agriculture and Industrial Production,20.328605,22.257143,10.690355


In [26]:
sql_2 = """SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under_30) as percent_under_30,
    AVG(percent_over_60) as percent_over_60
FROM wha_data
GROUP BY industry
ORDER BY percent_female DESC;
"""

cursor8.execute(sql_2)
wha_d= cursor8.fetchall()
df = pd.DataFrame(wha_d)
df

,0,1,2,3
0,Social Services,80.657143,25.745665,11.349570
1,Hospitals,76.427027,27.213793,16.489655
2,Leisure and Hospitality,53.804416,38.566343,11.544872
3,Business Sales and Services,50.632184,23.821752,12.465465
4,Public Administration,39.056738,21.015625,15.015385
5,Trade and Exchange,32.657258,29.108696,12.584034
6,Agriculture and Industrial Production,20.328605,22.257143,10.690355


## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]

In [27]:
last_data = wha_data.dropna(subset=["gender_balance", "premium"], how='all')

last_data = last_data.groupby(["gender_balance", "premium"]).size().sort_values(ascending=False)
ld = pd.DataFrame(last_data)
ld

0
gender_balance premium              
Mostly women   Partial coverage  333
Mostly men     Partial coverage  321
               Full coverage     293
Balanced       Partial coverage  271
Mostly women   Full coverage     267
Balanced       Full coverage     226
Mostly women   No coverage       107
Mostly men     No coverage        87
Balanced       No coverage        77

In [28]:
last_qry = """SELECT gender_balance, premium, COUNT(*)
FROM wha_data
GROUP BY gender_balance, premium
HAVING gender_balance is NOT NULL and premium is NOT NULL;"""

cursor8.execute(last_qry)
wha_d= cursor8.fetchall()
df = pd.DataFrame(wha_d)
df

,0,1,2
0,Balanced,Full coverage,226
1,Balanced,No coverage,77
2,Balanced,Partial coverage,271
3,Mostly men,Full coverage,293
4,Mostly men,No coverage,87
5,Mostly men,Partial coverage,321
6,Mostly women,Full coverage,267
7,Mostly women,No coverage,107
8,Mostly women,Partial coverage,333
